In [1]:
import cv2
import numpy as np

# Constants for heart rate detection
MIN_FACE_SIZE = 100
HEART_RATE_WINDOW_SIZE = 128
HEART_RATE_HISTORY_SIZE = 30
HEART_RATE_THRESHOLD = 20

# Initialize face detector and heart rate detector
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
heart_rate_history = np.zeros(HEART_RATE_HISTORY_SIZE)
heart_rate_index = 0

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Detect faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(MIN_FACE_SIZE, MIN_FACE_SIZE))

    # If a face is detected, calculate heart rate
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (HEART_RATE_WINDOW_SIZE, HEART_RATE_WINDOW_SIZE))
        heart_rate = np.fft.rfft(face_roi)[0].real
        heart_rate_scalar = np.mean(heart_rate) # Convert to scalar value
        heart_rate_history[heart_rate_index] = heart_rate_scalar
        heart_rate_index = (heart_rate_index + 1) % HEART_RATE_HISTORY_SIZE

        # Check if heart rate is above threshold
        stress_level = "Normal"
        if np.mean(heart_rate_history) - heart_rate_scalar > HEART_RATE_THRESHOLD:
            stress_level = "Stressed"

        # Draw heart rate and stress level on frame
        cv2.putText(frame, "Heart rate: {:.2f}".format(heart_rate_scalar), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(frame, "Stress level: {}".format(stress_level), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()